In [1]:
import pandas as pd
import numpy as np
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
con = sqlite3.connect(r'zippedData\im.db\im.db')

cur = con.cursor()

cur.execute("SELECT name from sqlite_master").fetchall()

[('movie_basics',),
 ('directors',),
 ('known_for',),
 ('movie_akas',),
 ('movie_ratings',),
 ('persons',),
 ('principals',),
 ('writers',)]

In [3]:
#directors has person_id,movie_id
#writers has movie_id,person_id
#known_for has person_id,movie_id
#movie_ratings movie_id,averagerating,numvotes
#persons has person_id,primary_name,birth_year,death_year,primary_profession
#principals has movie_id,person_id,category,job,characters
#movie_akas has movie_id,ordering,title,region,language,types,attributes,is_original_title
#movie_basics has movie_id,primary_title,original_title,start_year,runtime_minutes,genres

#persons,principals,movie_akas,movie_basics,movie_ratings are the only tables we need for all the columns in the db

combined = """SELECT *
                from principals
                JOIN movie_akas USING (movie_id)
                JOIN movie_basics USING (movie_id)
                JOIN movie_ratings USING (movie_id)
                JOIN persons USING (person_id);"""

combined_df = pd.read_sql_query(combined,con)
combined_df.head()



,movie_id,ordering,person_id,category,job,characters,ordering,title,region,language,...,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,primary_name,birth_year,death_year,primary_profession
0,tt0323808,10,nm0059247,editor,None,None,1,May Day,GB,None,...,The Wicker Tree,2011,96.0,"Drama,Horror",3.9,2328,Sean Barton,1944.0,NaN,"editor,editorial_department,assistant_director"
1,tt0323808,10,nm0059247,editor,None,None,2,Cowboys for Christ,GB,None,...,The Wicker Tree,2011,96.0,"Drama,Horror",3.9,2328,Sean Barton,1944.0,NaN,"editor,editorial_department,assistant_director"
2,tt0323808,10,nm0059247,editor,None,None,3,The Wicker Tree,GB,None,...,The Wicker Tree,2011,96.0,"Drama,Horror",3.9,2328,Sean Barton,1944.0,NaN,"editor,editorial_department,assistant_director"
3,tt0323808,10,nm0059247,editor,None,None,4,The Wicker Tree,None,None,...,The Wicker Tree,2011,96.0,"Drama,Horror",3.9,2328,Sean Barton,1944.0,NaN,"editor,editorial_department,assistant_director"
4,tt0323808,10,nm0059247,editor,None,None,5,Плетеное дерево,RU,None,...,The Wicker Tree,2011,96.0,"Drama,Horror",3.9,2328,Sean Barton,1944.0,NaN,"editor,editorial_department,assistant_director"


In [4]:
imdb_df = combined_df.copy()
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2422866 entries, 0 to 2422865
Data columns (total 24 columns):
 #   Column              Dtype  
---  ------              -----  
 0   movie_id            object 
 1   ordering            int64  
 2   person_id           object 
 3   category            object 
 4   job                 object 
 5   characters          object 
 6   ordering            int64  
 7   title               object 
 8   region              object 
 9   language            object 
 10  types               object 
 11  attributes          object 
 12  is_original_title   float64
 13  primary_title       object 
 14  original_title      object 
 15  start_year          int64  
 16  runtime_minutes     float64
 17  genres              object 
 18  averagerating       float64
 19  numvotes            int64  
 20  primary_name        object 
 21  birth_year          float64
 22  death_year          float64
 23  primary_profession  object 
dtypes: float64(5), int64(4),

In [5]:
#The columns with missing values
imdb_df.isna().any()[imdb_df.isna().any() == True]

job                   True
characters            True
region                True
language              True
types                 True
attributes            True
runtime_minutes       True
genres                True
birth_year            True
death_year            True
primary_profession    True
dtype: bool

In [6]:
#percentage null values
imdb_df.isna().mean()[imdb_df.isna().mean() > 0].sort_values(ascending=False)

death_year            0.982493
attributes            0.949760
language              0.855840
job                   0.725228
characters            0.606105
birth_year            0.564602
types                 0.402041
region                0.162510
runtime_minutes       0.039211
primary_profession    0.021650
genres                0.003738
dtype: float64

In [7]:
#dropping the columns with 40%+ null values
imdb_df.drop(labels=['death_year','attributes','language','job','characters','birth_year','types'],axis=1,inplace=True)

#checking columns with null values.
imdb_df.isna().mean()[imdb_df.isna().mean() > 0].sort_values(ascending=False)

region                0.162510
runtime_minutes       0.039211
primary_profession    0.021650
genres                0.003738
dtype: float64

In [31]:
#impute the null values using the mode values for the categorical columns. region,primary_profession,genres
#impute the null values using the mean values for the numerical colummns. runtime_minutes

# imdb_df['region'] = imdb_df[['region']].fillna(value=imdb_df['region'].mode(),inplace=True)
# imdb_df['primary_profession'] = imdb_df[['primary_profession']].fillna(value=imdb_df['primary_profession'].mode(),inplace=True)
# imdb_df['genres'] = imdb_df[['genres']].fillna(value=imdb_df['genres'].mode(),inplace=True)
# imdb_df['runtime_minutes'] = imdb_df[['runtime_minutes']].fillna(value=imdb_df['runtime_minutes'].mean(),inplace=True)

imdb_df.fillna( {'region':imdb_df['region'].mode(), 'primary_profession':imdb_df['primary_profession'].mode(),
                 'genres':imdb_df['genres'].mode(), 'runtime_minutes':imdb_df['runtime_minutes'].mean()},
                 inplace=True)

#checking columns with null values.
imdb_df.isna().mean()[imdb_df.isna().mean() > 0].sort_values(ascending=False)

c:\Users\franc\anaconda3\envs\learn-env\lib\site-packages\pandas\core\series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


region                0.162510
runtime_minutes       0.039211
primary_profession    0.021650
genres                0.003738
dtype: float64

In [32]:
#checking columns with null values.
imdb_df.isna().mean()[imdb_df.isna().mean() > 0].sort_values(ascending=False)


region                0.162510
runtime_minutes       0.039211
primary_profession    0.021650
genres                0.003738
dtype: float64

In [10]:
combined_df['attributes'].value_counts()

new title                      12668
alternative spelling           11384
original subtitled version      8184
literal English title           7661
complete title                  7513
                               ...  
TV listings title                 10
unauthorized video title           6
X-rated version                    6
first episode title                6
reissue titleshort version        4
Name: attributes, Length: 76, dtype: int64

In [11]:
rt_movie_info = pd.read_csv(r'zippedData/rt.movie_info.tsv/rt.movie_info.tsv',sep='\t')

rt_movie_info.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [12]:
# rt_movie_info['formatted_date'] = rt_movie_info['theater_date'].dt.strftime("%Y-%m-%d")
rt_movie_info['formatted_date'] = pd.to_datetime(rt_movie_info['theater_date'])
rt_movie_info.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio,formatted_date
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN,1971-10-09
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One,2012-08-17
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN,1996-09-13
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN,1994-12-09
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN,NaT


In [13]:
rt_movie_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              1560 non-null   int64         
 1   synopsis        1498 non-null   object        
 2   rating          1557 non-null   object        
 3   genre           1552 non-null   object        
 4   director        1361 non-null   object        
 5   writer          1111 non-null   object        
 6   theater_date    1201 non-null   object        
 7   dvd_date        1201 non-null   object        
 8   currency        340 non-null    object        
 9   box_office      340 non-null    object        
 10  runtime         1530 non-null   object        
 11  studio          494 non-null    object        
 12  formatted_date  1201 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(11)
memory usage: 158.6+ KB


In [14]:
rt_movie_info.isna().any()

id                False
synopsis           True
rating             True
genre              True
director           True
writer             True
theater_date       True
dvd_date           True
currency           True
box_office         True
runtime            True
studio             True
formatted_date     True
dtype: bool

In [15]:
rt_movie_info['rating'].value_counts()

R        521
NR       503
PG       240
PG-13    235
G         57
NC17       1
Name: rating, dtype: int64

In [16]:
rt_movie_info.groupby('genre').describe()

id               \
                                                   count         mean   
genre                                                                   
Action and Adventure                                19.0  1016.842105   
Action and Adventure|Animation|Art House and In...   1.0   135.000000   
Action and Adventure|Animation|Classics|Comedy|...   1.0  1156.000000   
Action and Adventure|Animation|Comedy                1.0   564.000000   
Action and Adventure|Animation|Comedy|Drama|Kid...   1.0  1418.000000   
...                                                  ...          ...   
Science Fiction and Fantasy                          6.0  1339.333333   
Science Fiction and Fantasy|Romance                  1.0  1908.000000   
Special Interest                                     1.0   636.000000   
Special Interest|Sports and Fitness                  1.0   265.000000   
Western                                              5.0  1260.000000   

                                                                        \
                                                           std     min   
genre                                                                    
Action and Adventure                                595.676484    70.0   
Action and Adventure|Animation|Art House and In...         NaN   135.0   
Action and Adventure|Animation|Classics|Comedy|...         NaN  1156.0   
Action and Adventure|Animation|Comedy                      NaN   564.0   
Action and Adventure|Animation|Comedy|Drama|Kid...         NaN  1418.0   
...                                                        ...     ...   
Science Fiction and Fantasy                         331.349765   835.0   
Science Fiction and Fantasy|Romance                        NaN  1908.0   
Special Interest                                           NaN   636.0   
Special Interest|Sports and Fitness                        NaN   265.0   
Western                                             429.774941   732.0   

                                                                             \
                                                        25%     50%     75%   
genre                                                                         
Action and Adventure                                 481.00   774.0  1593.5   
Action and Adventure|Animation|Art House and In...   135.00   135.0   135.0   
Action and Adventure|Animation|Classics|Comedy|...  1156.00  1156.0  1156.0   
Action and Adventure|Animation|Comedy                564.00   564.0   564.0   
Action and Adventure|Animation|Comedy|Drama|Kid...  1418.00  1418.0  1418.0   
...                                                     ...     ...     ...   
Science Fiction and Fantasy                         1138.25  1402.5  1615.0   
Science Fiction and Fantasy|Romance                 1908.00  1908.0  1908.0   
Special Interest                                     636.00   636.0   636.0   
Special Interest|Sports and Fitness                  265.00   265.0   265.0   
Western                                              880.00  1468.0  1493.0   

                                                            
                                                       max  
genre                                                       
Action and Adventure                                1913.0  
Action and Adventure|Animation|Art House and In...   135.0  
Action and Adventure|Animation|Classics|Comedy|...  1156.0  
Action and Adventure|Animation|Comedy                564.0  
Action and Adventure|Animation|Comedy|Drama|Kid...  1418.0  
...                                                    ...  
Science Fiction and Fantasy                         1660.0  
Science Fiction and Fantasy|Romance                 1908.0  
Special Interest                                     636.0  
Special Interest|Sports and Fitness                  265.0  
Western                                             1727.0  

[299 rows x 8 columns]

In [17]:
rt_reviews = pd.read_csv(r'zippedData\rt.reviews.tsv\rt.reviews.tsv',sep='\t',encoding='unicode_escape') #,on_bad_lines='skip'

rt_reviews.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [18]:
rt_reviews['formatted_date'] = pd.to_datetime(rt_reviews['date'])
rt_reviews.head()

,id,review,rating,fresh,critic,top_critic,publisher,date,formatted_date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018",2018-11-10
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018",2018-05-23
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018",2018-01-04
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017",2017-11-16
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017",2017-10-12


In [19]:
rt_movie_info[rt_movie_info['id'] == 3]

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio,formatted_date
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One,2012-08-17


In [20]:
rt_reviews[rt_reviews['id'] == 3]

,id,review,rating,fresh,critic,top_critic,publisher,date,formatted_date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018",2018-11-10
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018",2018-05-23
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018",2018-01-04
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017",2017-11-16
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017",2017-10-12
...,...,...,...,...,...,...,...,...,...
158,3,Beyond its withering critique of contemporary ...,NaN,fresh,David Jenkins,0,Little White Lies,"May 25, 2012",2012-05-25
159,3,"Threatens to soar and to be important, but it ...",3/5,fresh,Dave Calhoun,1,Time Out,"May 25, 2012",2012-05-25
160,3,A parade of hollow didactic encounters.,NaN,rotten,Owen Gleiberman,1,Entertainment Weekly,"May 25, 2012",2012-05-25
161,3,[An] agonisingly self-conscious and meagre pie...,2/5,rotten,Peter Bradshaw,0,Guardian,"May 25, 2012",2012-05-25


In [21]:
rt_reviews.columns

Index(['id', 'review', 'rating', 'fresh', 'critic', 'top_critic', 'publisher',
       'date', 'formatted_date'],
      dtype='object')

#### Fresh Rating

In [22]:
rt_reviews['fresh'].value_counts()

fresh     33035
rotten    21397
Name: fresh, dtype: int64

In [23]:
rt_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              54432 non-null  int64         
 1   review          48869 non-null  object        
 2   rating          40915 non-null  object        
 3   fresh           54432 non-null  object        
 4   critic          51710 non-null  object        
 5   top_critic      54432 non-null  int64         
 6   publisher       54123 non-null  object        
 7   date            54432 non-null  object        
 8   formatted_date  54432 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(6)
memory usage: 3.7+ MB


In [24]:
rt_reviews.isna().any()

id                False
review             True
rating             True
fresh             False
critic             True
top_critic        False
publisher          True
date              False
formatted_date    False
dtype: bool

In [25]:
rt_reviews['id'].value_counts()

782     338
1067    275
1525    262
1777    260
1083    260
       ... 
28        1
102       1
348       1
476       1
1727      1
Name: id, Length: 1135, dtype: int64

In [26]:
movie_budgets_df = pd.read_csv(r'zippedData\tn.movie_budgets.csv\tn.movie_budgets.csv') #,on_bad_lines='skip'

movie_budgets_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [27]:
movie_gross_df = pd.read_csv(r'zippedData\bom.movie_gross.csv\bom.movie_gross.csv')
movie_gross_df.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [28]:
movie

NameError: name 'movie' is not defined